In [1]:
!pip -q install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00


In [10]:
!pip -q install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.6 MB/s eta 0:00:00


In [27]:
import os

os.environ["OPENAI_API_KEY"] = ""
%set_env OPENAI_API_KEY=

env: OPENAI_API_KEY=


In [11]:
import openai
from openai import OpenAI

from langchain.prompts.prompt import PromptTemplate

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd

toy_data = pd.read_csv("/content/drive/My Drive//CIS6200/toy_dataset.csv")
toy_data.head()

,title,url,abstract,authors,review_title,review,rating
0,Pre-Training by Completing Point Clouds,https://openreview.net/pdf?id=jPSYH47QSZL,There has recently been a flurry of exciting a...,"Hanchen Wang,Qi Liu,Xiangyu Yue,Joan Lasenby,M...",A good one,This paper proposes a better pre-trained prior...,"7: Good paper, accept"
1,Pre-Training by Completing Point Clouds,https://openreview.net/pdf?id=jPSYH47QSZL,There has recently been a flurry of exciting a...,"Hanchen Wang,Qi Liu,Xiangyu Yue,Joan Lasenby,M...",paper shows promising results using point clou...,The paper considers the problem of training ne...,"7: Good paper, accept"
2,Pre-Training by Completing Point Clouds,https://openreview.net/pdf?id=jPSYH47QSZL,There has recently been a flurry of exciting a...,"Hanchen Wang,Qi Liu,Xiangyu Yue,Joan Lasenby,M...",Limited novelty and weak improvements,The authors propose completing an occluded poi...,4: Ok but not good enough - rejection
3,Pre-Training by Completing Point Clouds,https://openreview.net/pdf?id=jPSYH47QSZL,There has recently been a flurry of exciting a...,"Hanchen Wang,Qi Liu,Xiangyu Yue,Joan Lasenby,M...","Since the idea itself is simple enough, the re...",The idea of this paper is simple but fascinati...,5: Marginally below acceptance threshold
4,GamePad: A Learning Environment for Theorem Pr...,https://openreview.net/pdf?id=r1xwKoR9Y7,"In this paper, we introduce a system called Ga...","Daniel Huang,Prafulla Dhariwal,Dawn Song,Ilya ...",An intriguing integration of ML and automated ...,Summary: This paper mixes automated theorem pr...,"7: Good paper, accept"


In [21]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)
def send_prompt(prompt):
  response = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      # model="gpt-4-0125-preview",
      model="gpt-3.5-turbo",
      temperature=1,
  )
  return response.choices[0].message.content

In [18]:
template = """
You are a professional machine learning conference reviewer who reviews a given paper and considers 4 criteria: **importance and novelty**, **potential reasons for acceptance**, **potential reasons for rejection**, and **suggestions for improvement**.

You just need to use the following JSON format for output, but don't output opinions that don't exist in the original reviews. if you're not sure, return an empty dict:

{
'Significance and novelty': List multiple items by using Dict, The key is a brief description of the item, and the value is a detailed description of the item.

'Potential reasons for acceptance': List multiple items by using Dict, The key is a brief description of the item, and the value is a detailed description of the item.

"Potential reasons for rejection": List multiple items by using Dict, The key is a brief description of the item, and the value is a detailed description of the item.

'Suggestions for improvement': List multiple items by using Dict, The key is a brief description of the item, and the value is a detailed description of the item.
}

The given paper is as follows:

"""

# PROMPT = PromptTemplate(input_variables=["input"], template=template)

In [12]:
# sample

paper_abstract = toy_data["abstract"][0]
paper_abstract

'There has recently been a flurry of exciting advances in deep learning models on point clouds. However, these advances have been hampered by the difficulty of creating labelled point cloud datasets: sparse point clouds often have unclear label identities for certain points, while dense point clouds are time-consuming to annotate. Inspired by mask-based pre-training in the natural language processing community, we propose a pre-training mechanism based point clouds completion. It works by masking occluded points that result from observations at different camera views. It then optimizes a completion model that learns how to reconstruct the occluded points, given the partial point cloud. In this way, our method learns a pre-trained representation that can identify the visual constraints inherently embedded in real-world point clouds. We call our method Occlusion Completion (OcCo). We demonstrate that OcCo learns representations that improve the semantic understandings as well as generali

In [19]:
prompt = template + paper_abstract
print(prompt)


You are a professional machine learning conference reviewer who reviews a given paper and considers 4 criteria: **importance and novelty**, **potential reasons for acceptance**, **potential reasons for rejection**, and **suggestions for improvement**.

You just need to use the following JSON format for output, but don't output opinions that don't exist in the original reviews. if you're not sure, return an empty dict:

{
'Significance and novelty': List multiple items by using Dict, The key is a brief description of the item, and the value is a detailed description of the item.

'Potential reasons for acceptance': List multiple items by using Dict, The key is a brief description of the item, and the value is a detailed description of the item.

"Potential reasons for rejection": List multiple items by using Dict, The key is a brief description of the item, and the value is a detailed description of the item.

'Suggestions for improvement': List multiple items by using Dict, The key is

In [22]:
response = send_prompt(prompt)
print(response)

{
"Significance and novelty": {
"Mask-based pre-training": "The use of mask-based pre-training in the context of point clouds is a novel approach that draws inspiration from the natural language processing community.",
"Occlusion Completion (OcCo)": "The proposal of OcCo as a method for pre-training on point clouds through completion of occluded points is a unique and innovative contribution to the field."
},
"Potential reasons for acceptance": {
"Increased semantic understanding": "The paper demonstrates that the proposed method improves semantic understanding of point clouds compared to previous techniques.",
"Generalization on downstream tasks": "The method shows improved generalization on downstream tasks, which is a crucial aspect of any machine learning model."
},
"Potential reasons for rejection": {},
"Suggestions for improvement": {
"Experimental validation": "Providing more thorough experimental validation on a wider range of datasets could strengthen the paper's claims and re

In [23]:
def get_gpt_response(paper_abstract):
  prompt = template + paper_abstract
  response = send_prompt(prompt)
  return response

toy_data["gpt-3.5-turbo-reviews"] = toy_data["abstract"].apply(get_gpt_response)

In [24]:
toy_data.head(5)

,title,url,abstract,authors,review_title,review,rating,gpt-3.5-turbo-reviews
0,Pre-Training by Completing Point Clouds,https://openreview.net/pdf?id=jPSYH47QSZL,There has recently been a flurry of exciting a...,"Hanchen Wang,Qi Liu,Xiangyu Yue,Joan Lasenby,M...",A good one,This paper proposes a better pre-trained prior...,"7: Good paper, accept","{\n ""Significance and novelty"": {\n ..."
1,Pre-Training by Completing Point Clouds,https://openreview.net/pdf?id=jPSYH47QSZL,There has recently been a flurry of exciting a...,"Hanchen Wang,Qi Liu,Xiangyu Yue,Joan Lasenby,M...",paper shows promising results using point clou...,The paper considers the problem of training ne...,"7: Good paper, accept","{\n""Significance and novelty"": {\n ""Use of ..."
2,Pre-Training by Completing Point Clouds,https://openreview.net/pdf?id=jPSYH47QSZL,There has recently been a flurry of exciting a...,"Hanchen Wang,Qi Liu,Xiangyu Yue,Joan Lasenby,M...",Limited novelty and weak improvements,The authors propose completing an occluded poi...,4: Ok but not good enough - rejection,"{\n ""Significance and novelty"": {\n ..."
3,Pre-Training by Completing Point Clouds,https://openreview.net/pdf?id=jPSYH47QSZL,There has recently been a flurry of exciting a...,"Hanchen Wang,Qi Liu,Xiangyu Yue,Joan Lasenby,M...","Since the idea itself is simple enough, the re...",The idea of this paper is simple but fascinati...,5: Marginally below acceptance threshold,"{\n""Significance and novelty"": {\n ""Mask-ba..."
4,GamePad: A Learning Environment for Theorem Pr...,https://openreview.net/pdf?id=r1xwKoR9Y7,"In this paper, we introduce a system called Ga...","Daniel Huang,Prafulla Dhariwal,Dawn Song,Ilya ...",An intriguing integration of ML and automated ...,Summary: This paper mixes automated theorem pr...,"7: Good paper, accept","{\n ""Significance and novelty"": {\n ""Explo..."


In [25]:
csv_file_path = '/content/toy_data_w_gpt_reviews.csv'  # Define your file path here
toy_data.to_csv(csv_file_path, index=False)

In [26]:
from google.colab import files
files.download(csv_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>